In [1]:
import tensorflow as tf

import numpy as np
import os
import time


In [2]:
import urllib.request

url = "https://raw.githubusercontent.com/CalvinDO/AITextGenerator/main/LawLanguageRNN/dataText.txt"
response = urllib.request.urlopen(url)
text = response.read().decode(encoding='utf-8')

# Take a look at the first 250 characters in text
print(text[:250])


# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')
print(vocab)


Buch 1
Allgemeiner Teil
Abschnitt 1
Personen
Titel 1
Natuerliche Personen, Verbraucher, Unternehmer

§ 1 Beginn der Rechtsfaehigkeit
Die Rechtsfaehigkeit des Menschen beginnt mit der Vollendung der Geburt.

§ 2 Eintritt der Volljaehrigkeit
Die Vollja
86 unique characters
['\t', '\n', ' ', '"', '&', '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '§', 'ß', '̈', '–']


In [3]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

ids = ids_from_chars(chars)
ids


chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


chars = chars_from_ids(ids)
chars

tf.strings.reduce_join(chars, axis=-1).numpy()


def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids


ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())


def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text


B
u
c
h
 
1


A
l
l
tf.Tensor(
[b'B' b'u' b'c' b'h' b' ' b'1' b'\n' b'A' b'l' b'l' b'g' b'e' b'm' b'e'
 b'i' b'n' b'e' b'r' b' ' b'T' b'e' b'i' b'l' b'\n' b'A' b'b' b's' b'c'
 b'h' b'n' b'i' b't' b't' b' ' b'1' b'\n' b'P' b'e' b'r' b's' b'o' b'n'
 b'e' b'n' b'\n' b'T' b'i' b't' b'e' b'l' b' ' b'1' b'\n' b'N' b'a' b't'
 b'u' b'e' b'r' b'l' b'i' b'c' b'h' b'e' b' ' b'P' b'e' b'r' b's' b'o'
 b'n' b'e' b'n' b',' b' ' b'V' b'e' b'r' b'b' b'r' b'a' b'u' b'c' b'h'
 b'e' b'r' b',' b' ' b'U' b'n' b't' b'e' b'r' b'n' b'e' b'h' b'm' b'e'
 b'r' b'\n' b'\n'], shape=(101,), dtype=string)
b'Buch 1\nAllgemeiner Teil\nAbschnitt 1\nPersonen\nTitel 1\nNatuerliche Personen, Verbraucher, Unternehmer\n\n'
b'\xc2\xa7 1 Beginn der Rechtsfaehigkeit\nDie Rechtsfaehigkeit des Menschen beginnt mit der Vollendung der Gebu'
b'rt.\n\n\xc2\xa7 2 Eintritt der Volljaehrigkeit\nDie Volljaehrigkeit tritt mit der Vollendung des 18. Lebensjahr'
b'es ein.\n\n\xc2\xa7\xc2\xa7 3 bis 6 (weggefallen)\n\n\xc2\xa7 7 Wohnsitz; Beg

In [28]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list("Tensorflow"))

dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())


# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset


Input : b'Buch 1\nAllgemeiner Teil\nAbschnitt 1\nPersonen\nTitel 1\nNatuerliche Personen, Verbraucher, Unternehmer\n'
Target: b'uch 1\nAllgemeiner Teil\nAbschnitt 1\nPersonen\nTitel 1\nNatuerliche Personen, Verbraucher, Unternehmer\n\n'


<_PrefetchDataset element_spec=(TensorSpec(shape=(128, 100), dtype=tf.int64, name=None), TensorSpec(shape=(128, 100), dtype=tf.int64, name=None))>

In [29]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 16

# Number of RNN units
rnn_units = 128


In [30]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs

    x = self.embedding(x, training=training)
    #removed:
    #if states is None:
    #  states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x


model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)


for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape,
          "# (batch_size, sequence_length, vocab_size)")

model.summary()


sampled_indices = tf.random.categorical(
    example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

sampled_indices


#print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
#print()
#print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())


loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
#print("Prediction shape: ", example_batch_predictions.shape,
#      " # (batch_size, sequence_length, vocab_size)")
#print("Mean loss:        ", example_batch_mean_loss)

tf.exp(example_batch_mean_loss).numpy()


(128, 100, 87) # (batch_size, sequence_length, vocab_size)


Model: "my_model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (128, 100, 256)             │          22,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ ((128, 100, 1024), (128,    │       3,938,304 │
│                                      │ 1024))                      │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (128, 100, 87)              │          89,175 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,049,751 (15.45 MB)

 Trainable params: 4,049,751 (15.45 MB)

 Non-trainable params: 0 (0.00 B)

86.960014

In [7]:
# ONLY RUN WHEN LOADING A PRE-TRAINED MODEL
checkpoint_path = "./training_checkpoints/ckpt_100"

model.load_weights(checkpoint_path)


ValueError: File format not supported: filepath=./training_checkpoints/ckpt_100. Keras 3 only supports V3 `.keras` and `.weights.h5` files, or legacy V1/V2 `.h5` files.

In [31]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9),
              metrics=['accuracy'], loss=loss)



# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 3

In [32]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/3
673/673 ━━━━━━━━━━━━━━━━━━━━ 6729s 10s/step - accuracy: 0.1612 - loss: 3.4201
Epoch 2/3
673/673 ━━━━━━━━━━━━━━━━━━━━ 6535s 10s/step - accuracy: 0.2569 - loss: 2.7970
Epoch 3/3
673/673 ━━━━━━━━━━━━━━━━━━━━ 7227s 11s/step - accuracy: 0.3185 - loss: 2.4834


In [1]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')


NameError: name 'history' is not defined

In [2]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.5):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states





NameError: name 'tf' is not defined

In [27]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

start = time.time()
states = None
next_char = tf.constant([' '])
result = [next_char]

for n in range(600):
  next_char, states = one_step_model.generate_one_step(
      next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


 0plrsug ngeeinra, tizsnbr r3h nirerggi urnh ohu§nu. evsgtnedinaeaiue b dhnzlnmSe  lgeetetduun datrh wiessi UTeueem0esu uifetrDe ger.reeaf n uaegeeeuinB hoerslsewrmenw 8de idebigkrikvehork aekrdeai iuutSenbdrrre aemlnoz nhe n enruseiVe n i druiudotee uibuenrneFnntnuhitg nesnibu t2drten n hgte n zncfnn enefhacrrn nonue oe§cerne 3daserga reru uneA ßArs1rklk1ce  noAnere nihnuw g aitto reeetfr r dins,uatttfmurt*rn  tntneu ngtnup
Bc ris uddmal asrernuiaesen erVnnbdgcsr.i Se sdiKnrgAette nhvenibsahohussrusn2ngee eerriteDne O u,fnrasA het  l ag i toBe sdesLr,afefts  euehe wdde rtur easa2ueernie nez entPsn i1no erhn gted ef relhnsitati duxaeinntoa ezsuu0 fgdenau eng n  e ur N siidaAdnuer gidnIlsenei5dfdre dlrnrabe entreSlget m7c a rnun1ikd rd eng3; eeiieacetsbknr rn)hizrues mucmiecch etaevD he bn
Znrhkn rdtdssreg rrrsode t7hncix pmfiaenBessnltm i
snsri erirtnenniianaMrnvleßr,r ceV i ueasrrunlmnduorcd§tGiz gn 5hcbnmiuean te  z erwiglaoeah5 fs e nnsre na hsndlege vagRus2tansu nd ge nieir Gnulen1

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step\assets


INFO:tensorflow:Assets written to: one_step\assets


In [ ]:
states = None
next_char = tf.constant([' '])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_reloaded.generate_one_step(
      next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))


 Abt. 2 Satz 1 +++)
:a+kt der Dritten in durch § 1310 Abs. 2 Satz 2, wenn sie dem Anfang der Hypothek, eine Weise als mit dem Monat sowohl der Eigentuemer das Vermoegen uebernames Teilhabers abhaengt. Der Markei, so kann er nur unter den Erben oder eines Mangelhafter Ersicherung einer Vormunks, die den Vertragschliessenden oder des Mietverhaeltnisses zwischen der zweiter Dritter insoweit
(1) Wei einem eigenheit eine notarielle fuer die Lieferung einer Hyyothek, die zuzuegel hierung untirure, wilde gilt die Vorschrift des § 1378 Abs. 1 hinverlassenen Unterventsprich. Der Kuendigungspreis sechs Monate und ueber die Ehegatten anzubichen, es sei denn, dass, nach der Reiseveranstalter nach solchen dem in der Regel vorbiegen. Das Gesamtgut gekoendig anerklaeren, es sei denn, dass er zur Leistung bewinnt oder in den Faellen des § 144 entsprechende Anwendung, wenn die Fehlerhaftigkeit nicht vorziehen, gissanspauchsichtigt. Ist die Auflage sowie leendente kann nicht flei von Umpauf-Vollsichtlic